In [ ]:
import os
import subprocess
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import sys

In [ ]:
torch.mps.empty_cache()

# Check if MPS (Metal Performance Shaders) is available and set the device accordingly
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
# device = torch.device("cpu")

# Load tokenizer and model from Hugging Face Hub and move model to the MPS device if available
tokenizer = AutoTokenizer.from_pretrained("LLM4Binary/llm4decompile-6.7b-v1.5")
model = AutoModelForCausalLM.from_pretrained("LLM4Binary/llm4decompile-6.7b-v1.5")

# Convert model to FP16 to reduce memory usage
model = model.half().to(device)

In [ ]:
# Define the workspace directory within the repository for intermediate files
WORKSPACE_DIR = "decompile_workspace"

# Ensure the workspace directory exists
os.makedirs(WORKSPACE_DIR, exist_ok=True)

In [ ]:
def extract_function_asm(disassemble_path, function_name):
    # Extract the specified function's assembly
    input_asm = ''
    with open(disassemble_path, 'r') as f:
        asm = f.read()
        if f'<{function_name}>:' not in asm:
            raise ValueError(f"Function {function_name} not found in the assembly.")
        # Isolate the assembly for the specified function
        asm = f'<{function_name}>:' + asm.split(f'<{function_name}>:')[-1].split('\n\n')[0]
        asm_clean = ""
        for line in asm.splitlines():
            if len(line.split("\t")) < 3 and '00' in line:
                continue
            # Remove binary codes and comments
            asm_clean += "\t".join(line.split("\t")[2:]).split("#")[0].strip() + "\n"
    input_asm = asm_clean.strip()
    return input_asm

def compile(target: str, c_code_path: str, binary_path: str):
    if target == "mac":
        # Compile locally on macOS
        compile_command = f"gcc -o {binary_path}.o {c_code_path} -lm"
        subprocess.run(compile_command, shell=True, check=True)
    else:
        # Use Docker for cross-platform compilation (Linux x86_64)
        docker_image = "gcc_linux_x86_64:latest"
        container_name = "c_code_compiler"
        
        # Build Docker image targeting linux/amd64 architecture
        docker_build_command = (
            f"docker buildx build --platform linux/amd64 "
            f"-t {docker_image} . --load"
        )
        docker_run_command = (
            f"docker run --rm --name {container_name} "
            f"-v {os.getcwd()}:/workspace {docker_image} "
            f"gcc -o /workspace/{binary_path}.o /workspace/{c_code_path} -lm"
        )
        
        # Build Docker image
        subprocess.run(docker_build_command, shell=True, check=True)
        # Compile using Docker
        subprocess.run(docker_run_command, shell=True, check=True)

def disassemble_binary(binary_path, function_name, target: str):
    # Disassemble the binary directly
    disassemble_path = os.path.join(WORKSPACE_DIR, "disassembled_code.asm")
    
    if target == "mac":
        # Disassemble locally
        disassemble_command = ["objdump", "-d", binary_path]
        result = subprocess.run(disassemble_command, capture_output=True, text=True, check=True)
    else:
        # Use Docker for disassembly
        docker_image = "gcc_linux_x86_64:latest"
        container_name = "disassembler"
        
        docker_run_command = (
            f"docker run --rm --name {container_name} "
            f"-v {os.getcwd()}:/workspace {docker_image} "
            f"objdump -d /workspace/{binary_path}.o"
        )
        result = subprocess.run(docker_run_command, shell=True, capture_output=True, text=True, check=True)

    # Save disassembled code to a file for inspection
    with open(disassemble_path, "w") as f:
        f.write(result.stdout)
        
    input_asm = extract_function_asm(disassemble_path, function_name)
    
    return input_asm


def compile_and_disassemble_c_code(c_code_path, function_name):
    binary_path = os.path.join(WORKSPACE_DIR, "compiled_binary")
    target = "linux"
    
    if target == "mac":
        function_name = "_" + function_name

    # Compile the C code
    compile(target, c_code_path, binary_path)

    # Disassemble the binary
    input_asm = disassemble_binary(binary_path, function_name, target)
    
    return input_asm

def decompile_with_llm4decompile(disassembled_code):
    # Tokenize the input text and add attention mask
    inputs = tokenizer(disassembled_code, return_tensors="pt", padding=True).to(device)

    # Setting pad_token_id to eos_token_id, since this is the default behavior for many generative models
    model.config.pad_token_id = model.config.eos_token_id

    # Print the number of tokens in the input
    print(f"Number of tokens in input: {inputs.input_ids.size(1)}")
    
    # Generate decompiled code with attention mask
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=200
    )
    decompiled_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Save the decompiled code for inspection
    decompiled_path = os.path.join(WORKSPACE_DIR, "decompiled_code.c")
    with open(decompiled_path, "w") as f:
        f.write(decompiled_code)
    
    return decompiled_code

In [ ]:
def process_file(input_path, function_name):
    # Copy the C code or binary to the workspace for reference
    input_basename = os.path.basename(input_path)
    workspace_input_path = os.path.join(WORKSPACE_DIR, input_basename)
    if not os.path.exists(workspace_input_path):
        os.system(f"cp {input_path} {workspace_input_path}")
    
    try:
        if input_path.endswith(".c"):
            print("Input detected as C code. Compiling and disassembling...")
            disassembled_code = compile_and_disassemble_c_code(input_path, function_name)
        else:
            print("Input detected as binary. Disassembling...")
            disassembled_code = disassemble_binary(input_path, function_name)
        
        print("\nDisassembled Code:")
        print(disassembled_code)
        
        prompt = f"# This is the assembly code:\n{disassembled_code}\n# What is the source code?\n"
        
        # Decompile with LLM4Decompile
        print("\nDecompiling...")
        decompiled_code = decompile_with_llm4decompile(prompt)
        
        # Remove input from the decompiled code
        decompiled_code = decompiled_code[len(prompt):].strip()
        
        print("\nDecompiled Code:")
        print(decompiled_code)
    
    except subprocess.CalledProcessError as e:
        print(f"Error during processing: {e}")

In [ ]:
c_code_path = "c_code/fibonacci.c"
function_name = "func0"
process_file(c_code_path, function_name)